In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from datasets import load_dataset
import tiktoken

In [2]:
dataset = load_dataset("roneneldan/TinyStories")
train = dataset["train"]["text"]

In [3]:
encoder = tiktoken.get_encoding("cl100k_base")

In [6]:
with open("train_data.bin", "wb") as file:
    for i, text in enumerate(train):
        encoding = encoder.encode(text)
        np.array(encoding, dtype=np.int32).tofile(file)
        
        print(f"\rSaving data {(i/len(train))*100:.2f}% complete.", end="")



Saving data 100.00% complete.

In [7]:
train_tokens = torch.tensor(np.fromfile("train_data.bin", dtype=np.int32), dtype=torch.int32)

In [ ]:
class Transformer(nn.Model):
    def __init__(self, vocab_size):
        super().init()

        self.embeddings = nn.Embedding(vocab_size, 4)
        
        # self.query = nn.Linear(4,4)
        # self.key   = nn.Linear(4,4)
        # self.value = nn.Linear(4,4)
        
        self.self_attention = nn.Linear(4,4)
    def forward(x, self):
        # Q = query(x)
        # K = key(x)
        # V = value(x)
        # similarities = Q*K